<a href="https://colab.research.google.com/github/jkf87/unsloth-grpo-kr/blob/main/%5B%ED%95%9C%EA%B8%80%ED%99%94%5D_Llama3_1_(8B)_GRPO_ipynb%EC%9D%98_%EB%B0%B0%ED%8F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"*Runtime*"을 누르고 **무료** Tesla T4 Google Colab 인스턴스에서 "*Run all*"을 눌러 실행하세요!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> 도움이 필요하시면 Discord에 참여하세요 + ⭐ <i><a href="https://github.com/unslothai/unsloth">Github</a>에서 스타를 눌러주세요</i> ⭐
</div>

본인의 컴퓨터에 Unsloth를 설치하려면 Github 페이지의 설치 안내를 따라주세요 [여기를 클릭하세요](https://docs.unsloth.ai/get-started/installing-+-updating).

[데이터 준비](#Data), [학습 방법](#Train), [모델 실행 방법](#Inference), [저장 방법](#Save)을 배우게 됩니다.

모든 [모델 업로드](https://docs.unsloth.ai/get-started/all-our-models)와 [노트북](https://docs.unsloth.ai/get-started/unsloth-notebooks)은 문서를 참고하세요.

### News

추론 모델을 학습하는 방법에 대한 안내는 **[블로그 포스트](https://unsloth.ai/blog/r1-reasoning)를 참고하세요.** GRPO 노트북은 [@shxf0072](https://x.com/shxf0072/status/1886085377146180091), [@Teknium1](https://x.com/Teknium1/status/1885077369142337550), [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)의 영감을 받았습니다.

모든 [모델 업로드](https://docs.unsloth.ai/get-started/all-our-models)와 [노트북](https://docs.unsloth.ai/get-started/unsloth-notebooks)은 문서를 참고하세요.

### Installation

In [1]:
%%capture
# Colab에서 재시작 메시지 건너뛰기
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# 이 노트북을 로컬에서 실행하는 경우 `diffusers`도 설치해야 합니다
# !pip install diffusers
# 임시로 특정 TRL nightly version 설치
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

모든 함수 앞에 `PatchFastRL`을 사용하여 GRPO와 다른 RL 알고리즘을 패치하세요!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-17 11:12:00 __init__.py:190] Automatically detected platform cuda.


Llama 3.1 8B Instruct를 불러오고 매개변수를 설정하세요

In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # 더 긴 추론 과정을 위해 증가시킬 수 있음
lora_rank = 32 # 더 큰 랭크 = 더 똑똑하지만, 더 느림

model, tokenizer = FastLanguageModel.from_pretrained(
   model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
   max_seq_length = max_seq_length,
   load_in_4bit = True, # LoRA 16bit의 경우 False
   fast_inference = True, # vLLM 고속 추론 활성화
   max_lora_rank = lora_rank,
   gpu_memory_utilization = 0.6, # 메모리 부족 시 줄이세요
)

model = FastLanguageModel.get_peft_model(
   model,
   r = lora_rank, # 0보다 큰 숫자 선택! 추천: 8, 16, 32, 64, 128
   target_modules = [
       "q_proj", "k_proj", "v_proj", "o_proj",
       "gate_proj", "up_proj", "down_proj",
   ], # 메모리 부족 시 QKVO 제거
   lora_alpha = lora_rank,
   use_gradient_checkpointing = "unsloth", # 긴 문맥 파인튜닝 활성화
   random_state = 3407,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.5%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.16 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.01 GB. Also swap space = 6 GB.
INFO 02-17 11:12:21 config.py:542] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 02-17 11:13:39 model_runner.py:1562] Graph capturing finished in 36 secs, took 0.67 GiB
INFO 02-17 11:13:39 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 49.32 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 데이터 준비
<a name="Data"></a>

데이터 준비와 모든 보상 함수는 [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)의 코드를 활용합니다.

- `correctness_reward_func`: 응답에서 XML로 감싼 정답을 추출한 뒤, 실제 정답과 일치하면 2.0점을 부여합니다.
- `int_reward_func`: 응답에서 추출한 내용이 숫자(digit)로만 구성되어 있다면 0.5점을 부여합니다.
- `strict_format_reward_func`: 응답이 엄격한 XML 형식을 (특히 개행 문자 위치까지 정확히) 따른다면 0.5점을 부여합니다.
- `soft_format_reward_func`: 응답이 다소 유연한 XML 형식을 따른다면 0.5점을 부여합니다.
- `count_xml`: XML 태그의 정확한 등장 횟수와 태그 이후의 불필요한 내용에 따른 페널티를 적용하여 점수를 계산합니다.
- `xmlcount_reward_func`: 각 응답에 대해 count_xml 함수를 적용하여 보상 점수를 반환합니다.

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
  """
    - prompts: 원래의 질문들을 포함하는 리스트
    - completions: 모델이 생성한 응답들을 포함하는 리스트
    - answer: 올바른 정답을 포함하는 리스트

    함수 동작:
    1. completions에서 각 응답의 'content' 값을 추출합니다.
    2. prompts에서 질문 내용(마지막 항목의 'content')을 가져옵니다.
    3. extract_xml_answer() 함수를 이용해 각 응답에서 XML 형식의 정답 부분을 추출합니다.
    4. 디버깅용으로 질문, 정답, 응답, 추출된 정답을 출력합니다.
    5. 추출된 정답과 실제 정답을 zip()으로 순회하며, 일치하면 2.0의 보상을, 아니면 0.0의 보상을 리스트에 담아 반환합니다.
    """
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
  """
    - completions: 모델이 생성한 응답들을 포함하는 리스트

    함수 동작:
    1. completions에서 각 응답의 'content' 값을 추출합니다.
    2. 각 응답에서 XML 형식의 정답 부분을 추출합니다.
    3. 추출된 정답이 숫자로만 구성되어 있으면(문자열이 digit인지 검사) 0.5의 보상을, 아니면 0.0의 보상을 리스트에 담아 반환합니다.
    """

    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

#  응답이 엄격한 XML 형식(개행 문자 포함)을 따르는지 확인하는 함수

def strict_format_reward_func(completions, **kwargs) -> list[float]:
   """
    - completions: 모델이 생성한 응답들을 포함하는 리스트

    함수 동작:
    1. 엄격한 포맷의 정규표현식을 정의합니다.
       포맷:
           - <reasoning> 태그로 시작, 그 후 개행 문자와 내용, 다시 개행 후 </reasoning> 태그
           - 바로 이어서 <answer> 태그, 개행 문자와 내용, 개행 후 </answer> 태그로 끝남
    2. completions에서 각 응답의 'content' 값을 추출합니다.
    3. 각 응답이 정규표현식과 매치되는지 확인합니다.
    4. 매치되면 0.5의 보상을, 아니면 0.0의 보상을 리스트에 담아 반환합니다.
    """
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

#  응답이 다소 유연한 XML 형식을 따르는지 확인하는 함수

def soft_format_reward_func(completions, **kwargs) -> list[float]:

    """
    - completions: 모델이 생성한 응답들을 포함하는 리스트

    함수 동작:
    1. 좀 더 유연한 포맷의 정규표현식을 정의합니다.
       포맷:
           - <reasoning> 태그와 </reasoning> 태그 사이의 모든 내용 (non-greedy)
           - 임의의 공백(\s*) 이후 <answer> 태그와 </answer> 태그 사이의 모든 내용
    2. completions에서 각 응답의 'content' 값을 추출합니다.
    3. 각 응답이 정규표현식과 매치되는지 확인합니다.
    4. 매치되면 0.5의 보상을, 아니면 0.0의 보상을 리스트에 담아 반환합니다.
    """
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

#점수 계산하는 함수, XML 태그의 존재 여부 및 형식을 기반으로 점수를 계산
def count_xml(text) -> float:
  """
    - text: 모델의 응답 문자열

    함수 동작:
    1. 초기 점수를 0.0으로 설정합니다.
    2. '<reasoning>\n' 태그가 정확히 1번 등장하면 0.125점 추가.
    3. '\n</reasoning>\n' 태그가 정확히 1번 등장하면 0.125점 추가.
    4. '\n<answer>\n' 태그가 정확히 1번 등장하면 0.125점 추가하며,
       - 만약 이 태그 이후 추가적인 내용(예: 불필요한 텍스트)이 있다면, 그 길이에 비례하여 페널티(0.001점씩)를 차감.
    5. '\n</answer>' 태그가 정확히 1번 등장하면 0.125점 추가하며,
       - 태그 이후 내용의 길이에 따라(맨 처음 문자 제외) 페널티를 부여.
    6. 최종 계산된 점수를 반환합니다.

    이 함수는 XML 형식의 주요 태그들이 정확하게 한 번씩 등장하는지, 그리고 태그 이후 불필요한 내용이 있는지에 따라 보정된 점수를 산출합니다.
    """

    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

# 위에서 count_xlm 함수로 completions 적용해서 보상 계산
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
  """
    - completions: 모델이 생성한 응답들을 포함하는 리스트

    함수 동작:
    1. completions에서 각 응답의 'content' 값을 추출합니다.
    2. 각 응답 텍스트에 대해 count_xml() 함수를 적용하여 점수를 산출합니다.
    3. 계산된 점수를 리스트 형태로 반환합니다.
    """
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### 모델학습

이제 GRPO 트레이너와 모든 설정을 준비합니다.

In [6]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
   use_vllm = True, # 고속 추론을 위해 vLLM 사용!
   learning_rate = 5e-6,
   adam_beta1 = 0.9,
   adam_beta2 = 0.99,
   weight_decay = 0.1,
   warmup_ratio = 0.1,
   lr_scheduler_type = "cosine",
   optim = "paged_adamw_8bit",
   logging_steps = 1,
   bf16 = is_bfloat16_supported(),
   fp16 = not is_bfloat16_supported(),
   per_device_train_batch_size = 1,
   gradient_accumulation_steps = 1, # 더 부드러운 학습을 위해 4로 증가
   num_generations = 6, # 메모리 부족 시 감소
   max_prompt_length = 256,
   max_completion_length = 200,
   # num_train_epochs = 1, # 전체 학습을 위해 1로 설정
   max_steps = 10,
   save_steps = 10,
   max_grad_norm = 0.1,
   report_to = "none", # Weights & Biases 사용 가능
   output_dir = "outputs",
)

이제 트레이너를 실행해봅시다! 위로 스크롤하면 보상 테이블이 보일 것입니다. 목표는 `reward` 열의 값이 증가하는 것입니다!

어떤 동작이 나타나기까지 150~200 스텝 정도 기다려야 할 수 있습니다. 처음 100 스텝 동안은 아마도 0의 보상을 받을 것입니다. 기다려 주세요!

| 스텝 | 학습 손실 | 보상      | 보상 표준편차 | 완료 길이   | KL 발산  |
|------|-----------|-----------|---------------|-------------|----------|
| 1    | 0.000000  | 0.125000  | 0.000000     | 200.000000  | 0.000000 |
| 2    | 0.000000  | 0.072375  | 0.248112     | 200.000000  | 0.000000 |
| 3    | 0.000000  | -0.079000 | 0.163776     | 182.500000  | 0.000005 |

In [7]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 10
 "-____-"     Number of trainable parameters = 83,886,080


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
Let's break it down step by step.

Normal price of 1 ticket: $40
Normal price of 11 tickets (first 10 tickets): $40 * 10 = $400
First 10 tickets (with 10% discount): 10 tickets * $40 = $400, since you don't need a discount on the first 10 tickets.

First we need to find the price of 12 - 10 tickets = 2 tickets at discounted price
The total number of un-discounted tickets is 10, and the total number of tickets bought is 12, thus 12  -  10 = 2 tickets that have a 5% discount.

Price of 2 tickets with 5% discount: 2 * ($40 * 5%) = $40 * 10 = $4
Price of the last 2 tickets: $40 - (price of 2 tickets with 5% discount): $40 - $4 = 
Extracted:
Let's break it down step by step.

Normal price of 1 ticket: $40
Normal price of 11 tickets (first 10 tickets): $40 * 10 = $400
First 10 ti

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,184.333344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,198.333344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.114333,0.280058,181.333344,0.000000,-0.114333,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.394500,0.966324,170.500000,0.000777,-0.022167,0.000000,0.000000,0.083333,0.333333
5,0.000100,0.083333,0.204124,121.000000,0.001505,0.000000,0.000000,0.000000,0.083333,0.000000
6,0.000000,0.437500,1.011651,187.333344,0.000220,0.020833,0.000000,0.000000,0.083333,0.333333
7,0.000000,0.000000,0.000000,193.500000,0.000334,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.287333,0.907825,168.000000,0.000652,-0.129333,0.000000,0.000000,0.083333,0.333333
9,0.000000,0.437500,1.011651,172.166672,0.000369,0.020833,0.000000,0.000000,0.083333,0.333333
10,0.000000,0.000000,0.000000,200.000000,0.000280,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
To determine the monthly payment for each option, we need to calculate the total cost and then determine how to divide it over 20 years.

The total cost of the house is $480,000.
The total cost of the trailer is $120,000.

To find the monthly payment, we'll use the following formula: 
Monthly payment = Total cost / (number of payments)
The number of payments is 20 years * 12 months/year = 240 months

For the house: 
Monthly payment = $480,000 / 240 months
Monthly payment = $2000

For the trailer: 
Monthly payment = $120,000 / 240 months
Monthly payment = $500

To find the difference, we subtract the monthly payment for the trailer from the monthly payment for the house:
$2000 - $500 

TrainOutput(global_step=10, training_loss=1.6560379147811943e-05, metrics={'train_runtime': 319.8622, 'train_samples_per_second': 0.031, 'train_steps_per_second': 0.031, 'total_flos': 0.0, 'train_loss': 1.6560379147811943e-05})

<a name="Inference"></a>
### 추론
이제 방금 학습한 모델을 테스트해봅시다! 먼저, GRPO 학습 없이 모델을 시도해보겠습니다:

In [8]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.04s/it, est. speed input: 2.99 toks/s, output: 36.37 toks/s]


'Calculating pi is a complex task, and there are many methods to do so. Here\'s one way to calculate pi using the Monte Carlo method, a numerical method that uses random sampling to approximate a mathematical value. This method is simple to understand and implement, but it\'s not the most efficient way to calculate pi.\n\n**Monte Carlo Method**\n\nThe Monte Carlo method uses random sampling to estimate the ratio of the area of a circle to the area of a square. The ratio of these areas is pi.\n\nHere\'s a simple Python code snippet to calculate pi using the Monte Carlo method:\n\n```python\nimport random\nimport math\n\ndef calculate_pi(num_samples):\n    # Generate random points within a square of side length 2\n    points_inside_circle = 0\n    for _ in range(num_samples):\n        x = random.uniform(-1, 1)\n        y = random.uniform(-1, 1)\n        distance = math.sqrt(x**2 + y**2)\n        if distance <= 1:\n            points_inside_circle += 1\n\n    # Calculate pi using the rati

이제 GRPO로 학습한 LoRA를 사용해보겠습니다 - 먼저 LoRA를 저장합니다!

In [9]:
model.save_lora("grpo_saved_lora")

이제 LoRA를 불러와서 테스트해봅시다:

In [10]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.22s/it, est. speed input: 4.61 toks/s, output: 34.20 toks/s]


'Unfortunately, calculating pi to an infinite number of decimal places is a complex task that cannot be accomplished with a straightforward formula. However, I can provide you with a common method to calculate pi using the Monte Carlo method, which is an approximation method.\n\n**Method: The Monte Carlo Method**\n\nThe Monte Carlo method involves generating random numbers to approximate the value of pi. Here\'s a simple algorithm:\n\n1.  **Draw a square and a circle** with a point inside the square and a circle.\n2.  **Generate random points** inside the square.\n3.  **Count the ratio** of the number of points that fall inside the circle to the total number of points.\n\n**Code Example**\n\nHere\'s a Python code snippet that demonstrates this method:\n\n```python\nimport random\nimport math\n\ndef calculate_pi(num_points):\n    points_inside_circle = 0\n    for _ in range(num_points):\n        x = random.uniform(-1, 1)\n        y = random.uniform(-1, 1)\n        if math.sqrt(x**2 + y*

우리의 추론 모델이 훨씬 더 좋아졌습니다 - 1시간 정도만 학습했기 때문에 항상 정확하지는 않습니다 - 시퀀스 길이를 늘리고 더 오래 학습하면 더 좋아질 것입니다!

<a name="Save"></a>
### VLLM을 위한 float16 저장

우리는 `float16`로 직접 저장하는 것도 지원합니다. float16의 경우 `merged_16bit`를, int4의 경우 `merged_4bit`를 선택하세요. 대안으로 `lora` 어댑터도 지원합니다. Hugging Face 계정에 업로드하려면 `push_to_hub_merged`를 사용하세요! https://huggingface.co/settings/tokens 에서 개인 토큰을 얻을 수 있습니다.

In [13]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("JUNGU/llama3.1-8b-grpo-test", tokenizer, save_method = "merged_16bit", token = "hf_")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("JUNGU/llama3.1-8b-grpo-test", tokenizer, save_method = "merged_4bit", token = "hf_")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("JUNGU/llama3.1-8b-grpo-test", tokenizer, save_method = "lora", token = "hf_")

Unsloth: You are pushing to hub, but you passed your HF username = JUNGU.
We shall truncate JUNGU/llama3.1-8b-grpo-test to llama3.1-8b-grpo-test
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 23.39 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 22%|██▏       | 7/32 [00:00<00:01, 16.30it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:28<00:00,  1.10it/s]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/JUNGU/llama3.1-8b-grpo-test


### GGUF / llama.cpp 변환
이제 `GGUF` / `llama.cpp`로의 저장을 기본적으로 지원합니다! `llama.cpp`를 클론하고 기본적으로 `q8_0`으로 저장합니다. `q4_k_m`과 같은 모든 방법을 지원합니다. 로컬 저장에는 `save_pretrained_gguf`를, HF 업로드에는 `push_to_hub_gguf`를 사용하세요.

지원되는 양자화 방법들 ([위키 페이지](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)에서 전체 목록 확인):
* `q8_0` - 빠른 변환. 리소스 사용량이 높지만 일반적으로 허용 가능.
* `q4_k_m` - 추천. attention.wv와 feed_forward.w2 텐서의 절반에 Q6_K를 사용하고 나머지는 Q4_K 사용.
* `q5_k_m` - 추천. attention.wv와 feed_forward.w2 텐서의 절반에 Q6_K를 사용하고 나머지는 Q5_K 사용.



[**신규**] Ollama로 파인튜닝하고 자동 내보내기를 시도하려면 [Ollama 노트북](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)을 사용해보세요

In [14]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("JUNGU/llama3.1-8b-grpo-test", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("JUNGU/llama3.1-8b-grpo-test", tokenizer, quantization_method = "f16", token = "hf_)

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("JUNGU/llama3.1-8b-grpo-test", tokenizer, quantization_method = "q4_k_m", token = "hf_")

# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        "JUNGU/llama3.1-8b-grpo-test", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m"],
        token = "hf_",
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 26.14 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:21<00:00,  1.47it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at JUNGU/llama3.1-8b-grpo-test into bf16 GGUF format.
The output location will be /content/JUNGU/llama3.1-8b-grpo-test/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: llama3.1-8b-grpo-test
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-g

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/JUNGU/llama3.1-8b-grpo-test


이제 `model-unsloth.gguf` 파일이나 `model-unsloth-Q4_K_M.gguf` 파일을 llama.cpp나 Jan 또는 Open WebUI 같은 UI 기반 시스템에서 사용하세요. Jan은 [여기](https://github.com/janhq/jan)에서, Open WebUI는 [여기](https://github.com/open-webui/open-webui)에서 설치할 수 있습니다.

모든 과정이 끝났습니다! Unsloth에 대해 궁금한 점이 있다면 [Discord](https://discord.gg/unsloth) 채널이 있습니다! 버그를 발견했거나 최신 LLM 소식을 업데이트 받고 싶다면, 또는 도움이 필요하거나 프로젝트에 참여하고 싶다면 언제든지 Discord에 참여하세요!

기타 링크:
1. Llama 3.2 대화 노트북. [무료 Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Ollama로 파인튜닝 저장하기. [무료 노트북](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision 파인튜닝 - 방사선 사례. [무료 Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. DPO, ORPO, 지속적 사전 학습, 대화형 파인튜닝 등에 대한 노트북은 [문서](https://docs.unsloth.ai/get-started/unsloth-notebooks)를 참조하세요!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  도움이 필요하면 Discord에 참여하세요 + ⭐️ <i><a href="https://github.com/unslothai/unsloth">Github</a>에서 스타를 눌러주세요</i> ⭐️
</div>